In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import torch 
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoTokenizer
from tqdm import tqdm
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.classes.preprocessor import TextPreProcessor
from model1.spanemo.model import SpanEmo
from model1.spanemo.data_loader import DataClass
from fastprogress.fastprogress import format_time, master_bar, progress_bar
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seed = 12345678

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


In [3]:
paths_dict = {
    "cornell": "cornell_movie/predictions.csv",
    "iemocap": "iemocap/predictions.csv", #do ur stuff here
    "another": "another_df/predictions.csv", # do ur stuff here
}

df_dict = {
    "cornell": pd.read_csv(paths_dict["cornell"]),
    "iemocap": None,
    "another": None,
}

text_dict = {
    "cornell": np.array(df_dict["cornell"]["utterance text"]),
    "iemocap": np.array(0) # whatever
}

dataset_name = "cornell"
data_path = paths_dict[dataset_name]
pred_df = df_dict[dataset_name]
text_arr = text_dict[dataset_name]

In [4]:
hyperparams = {
    'backbone':'bert-base-uncased',
    'max_length': 128, #maximum context length for the model
    'batch_size': 128,
}

In [5]:
dataset = DataClass(hyperparams, text_arr[0:32768], pred_mode=True)
data_loader = DataLoader(dataset,
                        batch_size=hyperparams['batch_size'],
                        shuffle=False
                        )

/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter_2018 - 1grams ...


PreProcessing dataset ...:   0%|          | 0/32768 [00:00<?, ?it/s]/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
PreProcessing dataset ...: 100%|██████████| 32768/32768 [00:27<00:00, 1184.30it/s]


In [6]:
model_path = "../model1/final_model.pt"
model = SpanEmo()
model.load_state_dict(torch.load(model_path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [7]:

model.to(device)
model.eval()

index = 0

In [8]:
with torch.no_grad():
    current_index = 0
    for step, batch in enumerate(progress_bar(data_loader, parent=None, leave=False)):
        if step < current_index:
            current_index += hyperparams['batch_size']
            continue
        num_rows, y_pred, logits = model.predict(batch, device)
        pred_df['prediction'][current_index: current_index + num_rows, :] = logits.cpu().numpy()
        if step % 10 == 0: pred_df.to_csv(data_path)
        current_index += num_rows

pred_df.to_csv(data_path)

/tmp/ipykernel_232140/2232872937.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['prediction'][current_index: current_index + num_rows, :] = logits.cpu().numpy()


KeyError: 'key of type tuple not found and not a MultiIndex'